#Árvore

#Implementação em Python de uma Estrutura de Árvore Binária Encadeada

Define-se uma classe concreta LinkedBinaryTree que implementa o TAD (Tipo Abstrato de Dados) de árvore binária através da subclasse BinaryTree.

Define-se uma classe Node simples e não pública para representar um nó, e uma classe Position pública que encapsula um nó. Fornece-se um utilitário validate para verificar robustamente a validade de uma instância de posição dada ao desencapsulá-la, e um utilitário make position para encapsular um nó como uma posição a ser retornada ao chamador.

 Por formalidade, a nova classe Position é declarada para herdar diretamente de BinaryTree. Tecnicamente, a definição da classe BinaryTree não declara formalmente tal classe aninhada; ela a herda trivialmente de Tree. Um benefício menor deste design é que nossa classe de posição herda o método especial \__ne\__ para que a sintaxe p != q seja derivada apropriadamente em relação a \__eq\__.

Nossa definição de classe continua, com um construtor e com implementações concretas para os métodos que permanecem abstratos nas classes Tree e BinaryTree. O construtor cria uma árvore vazia inicializando raiz para None e tamanho para zero. Esses métodos acessores são implementados com uso cuidadoso dos utilitários validate e make position para proteger contra casos de limite.

In [ ]:
class Arvore:
    """Classe base abstrata representando uma estrutura de árvore."""
    class Posicao:
        """Uma abstração representando a localização de um único elemento."""
        def elemento(self):
            """Retorna o elemento armazenado nesta posição."""
            raise NotImplementedError('deve ser implementado pela subclasse')

        def __eq__(self, outro):
            """Retorna True se outra posição representa a mesma localização."""
            raise NotImplementedError('deve ser implementado pela subclasse')

    def raiz(self):
        """Retorna a posição representando a raiz da árvore (ou None se vazia)."""
        raise NotImplementedError('deve ser implementado pela subclasse')

    def pai(self, p):
        """Retorna a posição representando o pai de p (ou None se p for a raiz)."""
        raise NotImplementedError('deve ser implementado pela subclasse')

    def num_filhos(self, p):
        """Retorna o número de filhos que a posição p tem."""
        raise NotImplementedError('deve ser implementado pela subclasse')

    def __len__(self):
        """Retorna o número total de elementos na árvore."""
        raise NotImplementedError('deve ser implementado pela subclasse')

    def e_raiz(self, p):
        """Retorna True se a posição p representar a raiz da árvore."""
        return self.raiz() == p

    def e_folha(self, p):
        """Retorna True se a posição p não tiver filhos."""
        return self.num_filhos(p) == 0

    def e_vazia(self):
        """Retorna True se a árvore estiver vazia."""
        return len(self) == 0

class ArvoreBinaria(Arvore):
    """Classe base abstrata representando uma estrutura de árvore binária."""

    def esquerda(self, p):
        """Retorna uma posição representando o filho esquerdo de p.
        Retorna None se p não tiver filho esquerdo.
        """
        raise NotImplementedError('deve ser implementado pela subclasse')

    def direita(self, p):
        """Retorna uma posição representando o filho direito de p.
        Retorna None se p não tiver filho direito.
        """
        raise NotImplementedError('deve ser implementado pela subclasse')

    def irmao(self, p):
        """Retorna uma posição representando o irmão de p (ou None se não tiver irmão)."""
        pai = self.pai(p)
        if pai is None:
            return None  # a raiz não tem irmão
        else:
            if p == self.esquerda(pai):
                return self.direita(pai)  # possivelmente None
            else:
                return self.esquerda(pai)  # possivelmente None

    def filhos(self, p):
        """Gera uma iteração de posições representando os filhos de p."""
        if self.esquerda(p) is not None:
            yield self.esquerda(p)
        if self.direita(p) is not None:
            yield self.direita(p)

class ArvoreBinariaLigada(ArvoreBinaria):
    """Implementação concreta de uma árvore binária usando uma estrutura ligada."""

    class No:
        """Classe leve e não pública para armazenar um nó."""
        __slots__ = '_elemento', '_pai', '_esquerda', '_direita'

        def __init__(self, elemento, pai=None, esquerda=None, direita=None):
            self._elemento = elemento
            self._pai = pai
            self._esquerda = esquerda
            self._direita = direita

    class Posicao(ArvoreBinaria.Posicao):
        """Uma abstração representando a localização de um único elemento."""

        def __init__(self, container, no):
            self._container = container
            self._no = no

        def elemento(self):
            return self._no._elemento

        def __eq__(self, outro):
            return type(outro) is type(self) and outro._no is self._no

    def _validar(self, p):
        """Retorna o nó associado, se a posição for válida."""
        if not isinstance(p, self.Posicao):
            raise TypeError('p deve ser do tipo Posicao adequado')
        if p._container is not self:
            raise ValueError('p não pertence a este container')
        if p._no._pai is p._no:  # convenção para nós desativados
            raise ValueError('p não é mais válido')
        return p._no

    def _fazer_posicao(self, no):
        """Retorna uma instância de Posicao para um dado nó (ou None se não houver nó)."""
        return self.Posicao(self, no) if no is not None else None

    def __init__(self):
        """Cria uma árvore binária inicialmente vazia."""
        self._raiz = None
        self._tamanho = 0

    def __len__(self):
        """Retorna o número total de elementos na árvore."""
        return self._tamanho

    def raiz(self):
        """Retorna a posição da raiz da árvore (ou None se a árvore estiver vazia)."""
        return self._fazer_posicao(self._raiz)

    def pai(self, p):
        """Retorna a posição do pai de p (ou None se p for a raiz)."""
        no = self._validar(p)
        return self._fazer_posicao(no._pai)

    def esquerda(self, p):
        """Retorna a posição do filho esquerdo de p (ou None se não houver filho esquerdo)."""
        no = self._validar(p)
        return self._fazer_posicao(no._esquerda)

    def direita(self, p):
        """Retorna a posição do filho direito de p (ou None se não houver filho direito)."""
        no = self._validar(p)
        return self._fazer_posicao(no._direita)

    def adicionar_raiz(self, e):
        """Coloca o elemento e na raiz de uma árvore vazia e retorna a nova posição.
        Levanta um ValueError se a árvore não estiver vazia.
        """
        if self._raiz is not None: raise ValueError('A raiz já existe')
        self._tamanho = 1
        self._raiz = self.No(e)
        return self._fazer_posicao(self._raiz)

    def adicionar_esquerda(self, p, e):
        """Cria um novo filho esquerdo para a posição p, armazenando o elemento e.
        Retorna a posição do novo nó.
        Levanta um ValueError se a posição p for inválida ou se p já tiver um filho esquerdo.
        """
        no = self._validar(p)
        if no._esquerda is not None: raise ValueError('O filho esquerdo já existe')
        self._tamanho += 1
        no._esquerda = self.No(e, no)
        return self._fazer_posicao(no._esquerda)

    def adicionar_direita(self, p, e):
        """Cria um novo filho direito para a posição p, armazenando o elemento e.
        Retorna a posição do novo nó.
        Levanta um ValueError se a posição p for inválida ou se p já tiver um filho direito.
        """
        no = self._validar(p)
        if no._direita is not None: raise ValueError('O filho direito já existe')
        self._tamanho += 1
        no._direita = self.No(e, no)
        return self._fazer_posicao(no._direita)

#Testando!

In [ ]:
# Exemplo de uso da ArvoreBinariaLigada

arvore = ArvoreBinariaLigada()
raiz = arvore.adicionar_raiz(1)
esquerda = arvore.adicionar_esquerda(raiz, 2)
direita = arvore.adicionar_direita(raiz, 3)

print(f"Raiz: {raiz.elemento()}")
print(f"Filho esquerdo da raiz: {arvore.esquerda(raiz).elemento()}")
print(f"Filho direito da raiz: {arvore.direita(raiz).elemento()}")
print(f"Irmão do filho esquerdo: {arvore.irmao(esquerda).elemento()}")

Raiz: 1
Filho esquerdo da raiz: 2
Filho direito da raiz: 3
Irmão do filho esquerdo: 3


#Testando 2!

In [ ]:
if __name__ == "__main__":
    T = ArvoreBinariaLigada()
    r  = T.adicionar_raiz("A")          # raiz
    nB = T.adicionar_esquerda(r, "B")   # filho esquerdo
    nC = T.adicionar_direita(r,  "C")   # filho direito
    nD = T.adicionar_esquerda(nB, "D")

    print("Raiz:", r.elemento())                    # A
    print("Filhos de A:", [p.elemento() for p in T.filhos(r)])  # ['B', 'C']
    print("Irmão de B:", T.irmao(nB).elemento())    # C


Raiz: A
Filhos de A: ['B', 'C']
Irmão de B: C


# Conceitos de Orientação a Objetos Utilizados (em Árvores)

- **Classes e Objetos**: `Arvore`, `ArvoreBinaria`, `ArvoreBinariaLigada`, `Posicao`, e `No` são classes. `arvore`, `raiz`, `esquerda`, e `direita` são objetos.
  
- **Herança**: `ArvoreBinaria` herda de `Arvore` e `ArvoreBinariaLigada` herda de `ArvoreBinaria`.
  
- **Polimorfismo**: Métodos como `esquerda`, `direita`, `pai` são definidos em `ArvoreBinaria` e implementados em `ArvoreBinariaLigada`.

- **Encapsulamento**: Atributos privados dos nós (`_elemento`, `_pai`, `_esquerda`, `_direita`) são acessados apenas através de métodos.

- **Abstração**: As classes `Arvore` e `ArvoreBinaria` fornecem uma interface abstrata para a estrutura da árvore.

- **Métodos Abstratos**: Métodos como `esquerda` e `direita` são abstratos em `ArvoreBinaria` e devem ser implementados por subclasses.

- **Construtores**: O método `__init__` em `ArvoreBinariaLigada` inicializa a árvore.

- **Iteradores**: O método `filhos` em `ArvoreBinaria` usa geradores para iterar sobre os filhos de um nó.


#Implementação em Java (Baseado em Goodrich, M. T., Tamassia, R.,  Goldwasser, M. H. (2014). Data Structures and Algorithms in Java.)


```java
/* -------------- 1. Posição (cursor para nós) ----------------- */
public interface Position<E> {
    /** Devolve o elemento armazenado nesta posição. */
    E getElement();
}

/* -------------- 2. Interface geral de Árvore ----------------- */
public interface Tree<E> extends Iterable<E> {

    Position<E> root();
    Position<E> parent(Position<E> p);
    Iterable<Position<E>> children(Position<E> p);
    int numChildren(Position<E> p);

    int size();                     // nº de nós
    boolean isEmpty();

    /* utilidades típicas */
    default boolean isRoot(Position<E> p) { return p == root(); }
    default boolean isLeaf(Position<E> p) { return numChildren(p) == 0; }
}

/* ---------- 3. Classe abstrata com extras opcionais ---------- */
public abstract class AbstractTree<E> implements Tree<E> {

    /* profundidade em arestas */
    public int depth(Position<E> p) {
        return isRoot(p) ? 0 : 1 + depth(parent(p));
    }

    /* altura em arestas (versão recursiva) */
    public int height(Position<E> p) {
        int h = 0;
        for (Position<E> c : children(p))
            h = Math.max(h, 1 + height(c));
        return h;
    }

    /* altura da árvore inteira */
    public int height() {
        return isEmpty() ? 0 : height(root());
    }

    /* iteração por elementos em pré-ordem — simples mas útil */
    @Override
    public java.util.Iterator<E> iterator() {
        java.util.List<E> snapshot = new java.util.ArrayList<>();
        preorderSubtree(root(), snapshot);
        return snapshot.iterator();
    }
    private void preorderSubtree(Position<E> p, java.util.List<E> list) {
        if (p == null) return;
        list.add(p.getElement());
        for (Position<E> c : children(p))
            preorderSubtree(c, list);
    }
}

/* -------------- 4. Interface específica binária -------------- */
public interface BinaryTree<E> extends Tree<E> {

    Position<E> left(Position<E> p);
    Position<E> right(Position<E> p);

    /* irmão: usa apenas interface pública */
    default Position<E> sibling(Position<E> p) {
        Position<E> parent = parent(p);
        if (parent == null) return null;
        if (p == left(parent)) return right(parent);
        else                    return left(parent);
    }

    /* filhos em ordem esquerda→direita */
    @Override
    default Iterable<Position<E>> children(Position<E> p) {
        java.util.List<Position<E>> list = new java.util.ArrayList<>(2);
        if (left(p)  != null) list.add(left(p));
        if (right(p) != null) list.add(right(p));
        return list;
    }

    @Override
    default int numChildren(Position<E> p) {
        int n = 0;
        if (left(p)  != null) n++;
        if (right(p) != null) n++;
        return n;
    }
}

/* ---------- 5. Implementação concreta ligada ----------------- */
public class LinkedBinaryTree<E> extends AbstractTree<E>
                                 implements BinaryTree<E> {

    /* ---------- nó interno privado ---------- */
    protected static class Node<E> implements Position<E> {
        private E element;
        private Node<E> parent, left, right;
        Node(E e, Node<E> p, Node<E> l, Node<E> r) {
            element = e; parent = p; left = l; right = r;
        }
        /* implementação de Position */
        @Override public E getElement() { return element; }

        /* getters e setters package-private */
        Node<E> getParent()         { return parent; }
        Node<E> getLeft()           { return left; }
        Node<E> getRight()          { return right; }
        void setParent(Node<E> p)   { parent = p; }
        void setLeft(Node<E> l)     { left = l; }
        void setRight(Node<E> r)    { right = r; }
        void setElement(E e)        { element = e; }
    }

    /* ---------- campos da árvore ---------- */
    protected Node<E> root = null;
    private int size = 0;

    /* ---------- métodos de utilidade ---------- */
    protected Node<E> validate(Position<E> p) {
        if (!(p instanceof Node<?>))
            throw new IllegalArgumentException("Posição inválida");
    
        Node<E> node = (Node<E>) p;            // ② cast explícito
        if (node.getParent() == node)          // nó desactivado?
            throw new IllegalStateException("Posição não pertence mais à árvore");
        return node;
    }
    protected Position<E> makePosition(Node<E> node) {
        return (node == null) ? null : node;
    }

    /* ---------- implementação da interface ---------- */
    @Override public int size()           { return size; }
    @Override public boolean isEmpty()    { return size == 0; }
    @Override public Position<E> root()   { return makePosition(root); }

    @Override
    public Position<E> parent(Position<E> p) {
        Node<E> node = validate(p);
        return makePosition(node.getParent());
    }

    @Override
    public Position<E> left(Position<E> p) {
        Node<E> node = validate(p);
        return makePosition(node.getLeft());
    }

    @Override
    public Position<E> right(Position<E> p) {
        Node<E> node = validate(p);
        return makePosition(node.getRight());
    }

    /* ---------- operações de atualização (mutação) ---------- */
    public Position<E> addRoot(E e) {
        if (!isEmpty()) throw new IllegalStateException("Árvore já tem raiz");
        root = new Node<>(e, null, null, null);
        size = 1;
        return root;
    }

    public Position<E> addLeft(Position<E> p, E e) {
        Node<E> node = validate(p);
        if (node.getLeft() != null) throw new IllegalArgumentException("Filho esquerdo existe");
        Node<E> child = new Node<>(e, node, null, null);
        node.setLeft(child);
        size++;
        return child;
    }

    public Position<E> addRight(Position<E> p, E e) {
        Node<E> node = validate(p);
        if (node.getRight() != null) throw new IllegalArgumentException("Filho direito existe");
        Node<E> child = new Node<>(e, node, null, null);
        node.setRight(child);
        size++;
        return child;
    }

    /** Exemplo simples de remoção de folha; versões completas precisam lidar com + casos */
    public E removeLeaf(Position<E> p) {
        Node<E> node = validate(p);
        if (numChildren(p) > 0)
            throw new IllegalArgumentException("Não é folha");
        Node<E> parent = node.getParent();
        if (parent != null) {
            if (parent.getLeft() == node) parent.setLeft(null);
            else                          parent.setRight(null);
        } else {               // removendo a raiz
            root = null;
        }
        size--;
        node.setParent(node);  // marca como inválido
        return node.getElement();
    }
}

/* --------------------- 6. Demonstração --------------------- */
public class DemoTree {
    public static void main(String[] args) {
        LinkedBinaryTree<String> T = new LinkedBinaryTree<>();

        Position<String> a = T.addRoot("A");          // raiz
        Position<String> b = T.addLeft(a,  "B");      // filho esquerdo
        Position<String> c = T.addRight(a, "C");      // filho direito
        Position<String> d = T.addLeft(b,  "D");      // neto

        /* Percurso pré-ordem via iterator() */
        System.out.print("Pré-ordem: ");
        for (String e : T) System.out.print(e + " ");   // A B D C
        System.out.println();

        System.out.println("Altura: " + T.height());    // 2 arestas
        System.out.println("Tamanho: " + T.size());     // 4 nós

        /* Exemplo de remoção de folha */
        T.removeLeaf(d);
        System.out.println("Após remover D, tamanho = " + T.size()); // 3
    }
}

```

Resultado esperado:
```
Pré-ordem: A B D C
Altura: 2
Tamanho: 4
Após remover D, tamanho = 3
```